In [1]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime
import mysql.connector
# from PIL import ImageGrab


In [2]:
 
path = "./images"
images = []
classNames = []
myList = os.listdir(path)
print(myList)
for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])
    print(classNames)


In [3]:
 
def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList
 

In [9]:
def markAttendance(name):
    # Connect to the MySQL database
    try:
        db = mysql.connector.connect(
            host="localhost",      # Your MySQL host, e.g., localhost
            user="root",   # Your MySQL username
            password="", # Your MySQL password
            database="attendancedb"  # Your database name
        )
        cursor = db.cursor()

        # Open the CSV file
        with open('Attendance.csv', 'r+') as f:
            myDataList = f.readlines()
            nameList = []
            for line in myDataList:
                entry = line.split(',')
                nameList.append(entry[0])
            if name not in nameList:
                now = datetime.now()
                dtString = now.strftime('%H:%M:%S')
                f.writelines(f'\n{name},{dtString}')

                # Insert into the MySQL database
                sql = "INSERT INTO attendancet (name, time) VALUES (%s, %s)"
                val = (name, dtString)
                cursor.execute(sql, val)
                db.commit()

        print(f"Attendance marked for {name} at {dtString}")

    except mysql.connector.Error as err:
        print(f"Error: {err}")

    finally:
        if db.is_connected():
            cursor.close()
            db.close()

# Example usage
#markAttendance('John Doe')


In [ ]:
from datetime import datetime
import mysql.connector
import cv2
import numpy as np
import face_recognition
import os

def markAttendance(name):
    db = None
    try:
        db = mysql.connector.connect(
            host="localhost",
            user="root",  # Replace with your MySQL username
            password="",  # Replace with your MySQL password
            database="attendancedb"
        )
        cursor = db.cursor()

        now = datetime.now()
        dtString = now.strftime('%H:%M:%S')

        # Insert into the MySQL database
        sql = "INSERT INTO attendancet (name, time) VALUES (%s, %s)"
        val = (name, dtString)
        cursor.execute(sql, val)
        db.commit()

        print(f"Attendance marked for {name} at {dtString}")

    except mysql.connector.Error as err:
        print(f"Error: {err}")

    finally:
        if db is not None and db.is_connected():
            cursor.close()
            db.close()

def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

def load_images_from_folder(folder):
    images = []
    classNames = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
            classNames.append(os.path.splitext(filename)[0])
    return images, classNames

# Load images from a folder
folder_path = './images'
images, classNames = load_images_from_folder(folder_path)

if not images:
    print("No images found in the folder.")
else:
    encodeListKnown = findEncodings(images)
    print('Encoding Complete')

    cap = cv2.VideoCapture(0)

    while True:
        success, img = cap.read()
        if not success:
            print("Failed to capture image")
            break

        imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
        imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

        facesCurFrame = face_recognition.face_locations(imgS)
        encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)

        for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
            matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
            faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
            matchIndex = np.argmin(faceDis)

            if matches[matchIndex]:
                name = classNames[matchIndex].upper()
                y1, x2, y2, x1 = faceLoc
                y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
                cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
                markAttendance(name)

        cv2.imshow('Webcam', img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
